In [14]:
#!/usr/bin/env python
# coding: utf-8

# This code plots the parameter trace during a parameter estimation process.
# Note: This code plots the incomplete trace of samples because it reads sampels from ostOutput.txt.
# Only the parameter sets that improve the objective function in comparison with the previous parameter set are plotted.

# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

import os, sys, argparse, datetime, shutil
from glob import glob
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt 
import xarray as xr
import pandas as pd
from matplotlib.dates import DateFormatter
from tqdm import tqdm

# Function to extract a given setting from the configuration file
def read_from_control(control_file, setting):
    
    # Open 'control_active.txt' and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('|',1)[1].split('#',1)[0].strip() 
    # Return this value    
    return substring

# Function to extract a given setting from the summa and mizuRoute manager/control files
def read_from_summa_route_control(control_file, setting):

    # Open fileManager.txt or route_control and locate the line with setting
    with open(control_file) as ff:
        for line in ff:
            line = line.strip()
            if line.startswith(setting):
                break
    # Extract the setting's value
    substring = line.split('!',1)[0].strip().split(None,1)[1].strip("'")
    # Return this value    
    return substring

def is_number(s):
    try:
        float(s)
        return True 
    except (ValueError,AttributeError):
        return False 
                    
def read_param_sample_from_ostModel(ostModel):    
    sample_ids=[]
    objs = []
    sample_values = []
    
    data = np.loadtxt(ostModel, skiprows=1)
    sample_ids = data[:,0]
    objs = data[:,1]
    sample_values = data[:,2:]
    return  sample_ids,objs,sample_values

# Function to extract the param default values and bounds from basinParam and localParam.txt.
def read_basinParam_localParam(filename):
    param_names = []
    param_default = []
    param_min = []
    param_max =[]
    with open (filename, 'r') as f:
        for line in f:
            line=line.strip()
            if line and not line.startswith('!') and not line.startswith("'"):
                splits=line.split('|')
                if isinstance(splits[0].strip(), str):
                    param_names.append(splits[0].strip())
                    param_default.append(str_to_float(splits[1].strip()))
                    param_min.append(str_to_float(splits[2].strip()))
                    param_max.append(str_to_float(splits[3].strip()))
    return param_names, param_default, param_min, param_max

# Function to convert data from Fortran format to scientific format.
def str_to_float(data_str):
    if 'd' in data_str:
        x = data_str.split('d')[0]+'e'+data_str.split('d')[1]
        return float(x)
    else:
        return float(data_str)

# main
if __name__ == '__main__':
    
    # ----------------------------- Settings ------------------------------        
    # calib inputs
    root_path = '/home/h294liu/project/proj/5_summaCalib'  # root path where parameter estimation will be stored.
    domain_name = 'BowAtBanff' #'BowAtBanff_LA_calib' #'BowAtBanff'

    calib_basename = 'DDS' #SCE #GA #DDS #GLUE  
    default_model_folder = 'BowAtBanff_default' #BowAtBanff_LA' #'BowAtBanff_default'
    outFilePrefix = 'run1'

    # inpuths
    calib_output_path = os.path.join(root_path, domain_name+'_'+calib_basename)
    default_output_path = os.path.join(root_path, default_model_folder)
    direct_param_list = []
    
    # identify plot output path and file
    output_path = os.path.join(calib_output_path, 'analysis', '8_plot_param_trace_collect')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    ofile_fig = os.path.join(output_path, '%s_param_trace.png'%(calib_basename))   # output plot figure
    ofile_txt = os.path.join(output_path, '%s_best_summary.txt'%(calib_basename)) # output best param information
    ofile_param_txt = os.path.join(output_path, '%s_param_samples.txt'%(calib_basename)) # output all param information
    trialParamFile_temp = os.path.join(output_path, 'trialParam.temp.nc')         # temporary trialParam file   
   
    # --------------------------- End settings -----------------------------        
    
    # 1. Read deafault model information (control file, param, obj)    
    control_file = os.path.join(default_output_path, 'calib','control_active.txt')
    domain_name = read_from_control(control_file, 'domain_name')
    domain_path = os.path.join(root_path, domain_name)

    # hydrologic model path, settings, fileManager.txt, trialParam.nc.
    model_dst_path = read_from_control(control_file, 'model_dst_path')
    if model_dst_path == 'default':
        model_dst_path = os.path.join(domain_path, 'model')

    summa_settings_relpath = read_from_control(control_file, 'summa_settings_relpath')
    summa_settings_path = os.path.join(model_dst_path, summa_settings_relpath)

    summa_filemanager = read_from_control(control_file, 'summa_filemanager')
    summa_filemanager = os.path.join(summa_settings_path, summa_filemanager)
      
    trialParamFile = read_from_summa_route_control(summa_filemanager, 'trialParamFile')
    trialParamFile_priori = trialParamFile.split('.nc')[0] + '.priori.nc' # a priori param file
    trialParamFile_priori = os.path.join(summa_settings_path, trialParamFile_priori)
    
    # 2. Read parameter names and ranges from default model
    print('Read parameter names and ranges.')
    # (1) read parameter names from summa_parameter_bounds.txt
    # param_names is a list of params corresponding to multiplier, used to update trialParam file.
    param_names = list(np.loadtxt(os.path.join(default_output_path, 'calib', 'summa_parameter_bounds.txt'), 
                                  usecols=[0], dtype='str', delimiter=','))
    param_range = np.loadtxt(os.path.join(default_output_path, 'calib', 'summa_parameter_bounds.txt'), 
                             usecols=[2,3], delimiter=',')
    
    # (2) read parameter ranges from localParam.txt and basinParam.txt.
    basinParam = read_from_summa_route_control(summa_filemanager, 'globalGruParamFile')
    localParam = read_from_summa_route_control(summa_filemanager, 'globalHruParamFile')

    basinParam = os.path.join(summa_settings_path, basinParam)
    localParam = os.path.join(summa_settings_path, localParam)

    basin_param_names, basin_param_default, basin_param_min, basin_param_max = read_basinParam_localParam(basinParam)    
    local_param_names, local_param_default, local_param_min, local_param_max = read_basinParam_localParam(localParam)

    # 3. Get a more complete list of params, used for plot. 
    print('Get param name list of plot.')
    output_param_names = param_names.copy() 
    
#     # add soil parameters if soil parameters are included in param_names.
#     soil_params = ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity', 'theta_sat']
#     if any(soil_param in param_names for soil_param in soil_params):
#         for soil_param in soil_params:
#             if not soil_param in output_param_names:
#                 output_param_names.append(soil_param)
                
    # add canopy height parameters if thickness if included in param_names.
    canopyHeigh_params = ['heightCanopyBottom', 'heightCanopyTop']
    if 'thickness' in param_names:
        output_param_names.remove('thickness')
        for canopyHeigh_param in canopyHeigh_params:
            if not canopyHeigh_param in output_param_names:
                output_param_names.append(canopyHeigh_param)
            
    # sort output_param_names.
    output_param_names.sort()
    

    # 4. Read calib ostModel
    print('Read ostModel.')
    sample_ids,objs,multp_values = read_param_sample_from_ostModel(os.path.join(calib_output_path, 'OstModel.txt'))
    sample_ids_concat,objs_concat,multp_values_concat = sample_ids,objs,multp_values
        
    # identify the iteration/runthe best parameter sets
    best_obj = np.min(objs_concat)
    best_indices = np.argwhere(objs_concat == best_obj)
    
    if len(best_indices)==1:
        best_runs_str = str(int(sample_ids_concat[best_indices]))
    elif len(best_indices)>1:
        best_runs_str = ",".join(map(str,map(int, list(sample_ids_concat[best_indices]))))
        
    
    # 5. Update param values based on multipler value and param priori value. 
    print('Update parameter values based on multipler samples.')
    Nsample = len(sample_ids_concat)
    Nparam = len(output_param_names)
    output_param_values = np.zeros((Nsample, Nparam)) #store the updated param values in array

    with nc.Dataset(trialParamFile_priori, 'r') as src:

        # loop all multiplier samples to get corresponding parameter values       
        pbar = tqdm(total=Nsample)
        for iSample in range(Nsample):
            multp_iSample = multp_values_concat[iSample]           
        
            # --- copy trialParamFile_priori to be the base of trialParamFile.
            shutil.copy(trialParamFile_priori, trialParamFile_temp)

            # --- update param values in a template trialParam file and save new param values to array.
            with nc.Dataset(trialParamFile_temp, 'r+') as dst:

                for iParam in range(len(param_names)):
                    param_name = param_names[iParam]

                    # Part A: update all params except 'thickness'
                    if (param_name != 'thickness') and param_name in dst.variables.keys():  

                        # update param values
                        if not param_name in direct_param_list:# new_value = multipler * default_value
                            param_ma_priori = src.variables[param_name][:]               # priori param value mask array 
                            param_value     = param_ma_priori.data * multp_iSample[iParam]     # update param value mask array
                            dst.variables[param_name][:] = np.ma.array(param_value, \
                                                                       mask=np.ma.getmask(param_ma_priori), \
                                                                       fill_value=param_ma_priori.get_fill_value())
                        elif param_name in direct_param_list: # new_value = Ostrich value
                            param_ma_priori = src.variables[param_name][:]                          # priori param value mask array 
                            param_value     = np.ones_like(param_ma_priori.data) * multp_iSample[iParam]  # update param value mask array
                            dst.variables[param_name][:] = np.ma.array(param_value, \
                                                                       mask=np.ma.getmask(param_ma_priori),  \
                                                                       fill_value=param_ma_priori.get_fill_value())                   

#                         # if param is 'theta_sat', update other four soil variables using a priori param value fractions.
#                         if param_name == 'theta_sat':
#                             param_ma_priori  = src.variables[param_name][:]
#                             param_ma = dst.variables[param_name][:]

#                             for add_param in ['theta_res', 'critSoilWilting', 'critSoilTranspire', 'fieldCapacity']:
#                                 add_param_ma_priori  = src.variables[add_param][:]
#                                 fraction =  np.divide(add_param_ma_priori.data, param_ma_priori.data) # fraction based on priori variable values
#                                 add_param_value = param_ma.data * fraction
#                                 dst.variables[add_param][:]= np.ma.array(add_param_value,                                                                          mask=np.ma.getmask(add_param_ma_priori),                                                                          fill_value=add_param_ma_priori.get_fill_value())

                # After updating all parameters, update 'thickness' if it exists. 
                # Actually, use 'thickness' to calculate TopCanopyHeight.
                if 'thickness' in param_names:
                    tied_param_name   = 'heightCanopyBottom'
                    target_param_name = 'heightCanopyTop'

                    # update TopCanopyHeight
                    TH_param_ma_priori = src.variables[target_param_name][:]         # priori TopCanopyHeight mask array 
                    BH_param_ma_priori = src.variables[tied_param_name][:]           # priori BottomCanopyHeight mask array 
                    BH_param_ma        = dst.variables[tied_param_name][:]           # updated BottomCanopyHeight mask array
                    param_value        = BH_param_ma.data + \
                    (TH_param_ma_priori.data-BH_param_ma_priori.data)*multp_iSample[iParam]    # updated TopCanopyHeight values
                    dst.variables[target_param_name][:] = np.ma.array(param_value, \
                                                                      mask=np.ma.getmask(TH_param_ma_priori),  \
                                                                      fill_value=TH_param_ma_priori.get_fill_value())
            
                #  Part B: Extract output parameter values for the first GRU/HRU (for plot).
                # Be careful. Hard coded!
                for jParam in range(Nparam):
                    param_name = output_param_names[jParam]
                    param_ma = dst.variables[param_name][:]
                    output_param_values[iSample, jParam]=param_ma.data.flat[0]

            pbar.update(1)
        pbar.close()  
        
    # 6. Save the best param sample into text.
    print('Save the best parameter set')
    f = open(ofile_txt, "w")
    print('--- Best -KGE = %.4f. Occur %d time(s). \nAt run %s. \n'%(best_obj, len(best_indices), best_runs_str))
    f.write('Best -KGE = %.4f. Occur %d time(s). \nAt run %s. \n\n'%(best_obj, len(best_indices), best_runs_str))

    for param_index in range(Nparam):
        param_name = output_param_names[param_index]
        best_param_values = output_param_values[best_indices, param_index]
        # if there are multiple best param sets, and all best param values are the same, save a single param value.
        if len(best_indices)>1 and np.all(best_param_values == best_param_values[0]):
            print('--- %s = %.6f.'%(param_name, best_param_values[-1]))   
            f.write('%s = %.6f\n'%(param_name, best_param_values[-1]))

        # if best param values are different, save all param values.
        else: 
            best_param_values_str = ",".join(map(str,map(float, list(best_param_values))))
            print('--- %s = %s.'%(param_name, best_param_values_str))   
            f.write('%s = %s\n'%(param_name, best_param_values_str))    
    f.close()


    # 7. Save all param samples into text
    print('Save parameter traces.')
    f = open(ofile_param_txt, 'w')    
    firstLine = 'Run\tobj.function\t'+'\t'.join(output_param_names)+'\n'
    f.write(firstLine)        
    for iSample in range(Nsample):
        f.write('%d\t'%(sample_ids[iSample]))
        f.write('%.6E\t'%(objs_concat[iSample])) # obj = -KGE
        for iParam in range(Nparam):
            f.write('%.6E\t'%(output_param_values[iSample,iParam]))
        f.write('\n')
    f.close()
    
    # 8. Gather a priori param values and model performance for plot
    priori_param_values = np.zeros((Nparam,)) #store the a priori param values in array
    with nc.Dataset(trialParamFile_priori, 'r') as src:
        for iParam in range(Nparam):
            param_name = output_param_names[iParam]
            if param_name in src.variables.keys():  
                param_ma_priori    = src.variables[param_name][:]     # priori param value mask array 
                param_value_priori = param_ma_priori.data.flat[0]     # value of the first HRU/GRU 
                priori_param_values[iParam] = param_value_priori
            else:
                priori_param_values[iParam] = np.nan

    # model performance
#     stat_default = np.loadtxt(os.path.join(default_output_path, 'calib/trial_stats.txt'), delimiter='#', usecols=[0]) # KGE (not -KGE)
    obj_default = stat_default[0] * (-1) # -KGE 
                             

    # 9. Plot objective function and parameter trace.
    print('Plot parameter traces.')
    col_num = 4        
    row_num = int(np.ceil(Nparam/float(col_num)))
    
    fig, ax = plt.subplots(row_num,col_num, figsize=(3.0*col_num, 3.0*0.75*row_num), constrained_layout=True)
    fig.suptitle('%s'%(calib_basename), fontsize='large', fontweight='bold')
    
    dpi_value=80
    ms_sample = 1 # marker size for samples
    ms_highlight = 4 # marker size for highlight points   
    
    for i in range(row_num):
        for j in range(col_num):
            
            subplot_count = i*col_num + j
            param_index = subplot_count - 1
            
            if subplot_count <= Nparam: 
                
                if i==0 and j==0: 
                    # plot obj function 
                    ax[i,j].plot(sample_ids_concat, objs_concat, color='g',marker='^',
                                 linewidth=0.0, markersize=ms_sample) 
                    fig_count = i*col_num + j
                    title_str = '('+chr(ord('a') + subplot_count) +') ' + 'Objective function'   
                    y_label = '-KGE'

                    # plot a priori and best obj functions
                    ax[i,j].plot(sample_ids_concat[0], obj_default, 
                                 'D', markerfacecolor="none", markeredgecolor="k", markersize=ms_highlight); #darkorange
                    ax[i,j].plot(sample_ids_concat[best_indices], objs_concat[best_indices], 
                                 's', markerfacecolor="none", markeredgecolor="red", markersize=ms_highlight);
                    
                else:
                    # plot parameters
                    param_name = output_param_names[param_index]
                    ax[i,j].plot(sample_ids_concat, output_param_values[:, param_index], color='blue',marker='o',
                                 linewidth=0.0, markersize=ms_sample)
                    title_str = '('+chr(ord('a') + subplot_count) +') ' + param_name
                    y_label = 'Value' 

                    # plot the a priori and best points 
                    ax[i,j].plot(sample_ids_concat[0], priori_param_values[param_index], 
                                 'D', markerfacecolor="none", markeredgecolor="k", markersize=ms_highlight);
                    
                    best_param_values = output_param_values[best_indices, param_index]
                    ax[i,j].plot(sample_ids_concat[best_indices], best_param_values, 
                                 's', markerfacecolor="none", markeredgecolor="red", markersize=ms_highlight);
                                    
                # ylimit, axis, label, title
                if not (i==0 and j==0):
                    if param_name in param_names:
                        param_min = param_range[param_names.index(param_name)][0]
                        param_max = param_range[param_names.index(param_name)][1]
                    elif param_name in local_param_names:
                        param_min = local_param_min[local_param_names.index(param_name)]
                        param_max = local_param_max[local_param_names.index(param_name)]
                    elif param_name in basin_param_names:
                        param_min = basin_param_min[basin_param_names.index(param_name)]
                        param_max = basin_param_max[basin_param_names.index(param_name)]            
                    ax[i,j].set_ylim([param_min, param_max])

                ax[i,j].set_title(title_str, fontsize='medium', fontweight='semibold')
                ax[i,j].set_xlabel('Iterations', fontsize='medium')
                ax[i,j].set_ylabel(y_label, fontsize='medium')
                            
            # blank subplots           
            else: 
                # plot legend
                if subplot_count == Nparam+1: 
                    ax[i,j].set_frame_on(False)
                    ax[i,j].get_xaxis().set_visible(False)
                    ax[i,j].get_yaxis().set_visible(False)
                    ax[i,j].plot(np.nan, np.nan, '^', markerfacecolor="green", markeredgecolor='none', label = 'Objective function')
                    ax[i,j].plot(np.nan, np.nan, 'o', markerfacecolor="blue", markeredgecolor='none', label = 'Parameter sample')
                    ax[i,j].plot(np.nan, np.nan, 'D', markerfacecolor="none", markeredgecolor="k", markersize=ms_highlight, label='A priori value') #darkorange
                    ax[i,j].plot(np.nan, np.nan, 's', markerfacecolor="none", markeredgecolor="red", markersize=ms_highlight, label='Best value')
                    ax[i,j].legend(loc = 'center left')
                else:
                    ax[i,j].axis('off')

    plt.rc('xtick',labelsize='medium')
    plt.rc('ytick',labelsize='medium')                
    fig.savefig(ofile_fig, dpi=dpi_value)
    plt.close(fig)  
    if os.path.exists(trialParamFile_temp): 
        os.remove(trialParamFile_temp)         
  

  0%|          | 0/683 [00:00<?, ?it/s]

Read parameter names and ranges.
Get param name list of plot.
Read ostModel.
Update parameter values based on multipler samples.


100%|██████████| 683/683 [00:11<00:00, 58.26it/s]


Save the best parameter set
--- Best -KGE = -0.8997. Occur 2 time(s). 
At run 94,95. 

--- Fcapil = 0.017192.
--- aquiferBaseflowExp = 1.226019.
--- aquiferBaseflowRate = 0.041976.
--- frozenPrecipMultip = 1.298033.
--- heightCanopyBottom = 0.289034.
--- heightCanopyTop = 11.476940.
--- k_macropore = 0.045000.
--- k_soil = 0.000001.
--- qSurfScale = 13.780125.
--- routingGammaScale = 17739.930927.
--- routingGammaShape = 2.626733.
--- summerLAI = 5.182137,5.872815.
--- theta_sat = 0.423549.
Save parameter traces.
Plot parameter traces.


In [7]:
output_param_names

['Fcapil',
 'aquiferBaseflowExp',
 'aquiferBaseflowRate',
 'frozenPrecipMultip',
 'heightCanopyBottom',
 'heightCanopyTop',
 'k_macropore',
 'k_soil',
 'qSurfScale',
 'routingGammaScale',
 'routingGammaShape',
 'summerLAI',
 'theta_sat']

In [17]:
priori_param_values

array([6.00000000e-02, 2.00000000e+00, 1.00000000e-03, 1.00000000e+00,
       8.50000000e+00, 2.00000000e+01, 1.00000000e-03, 1.39472000e-06,
       5.00000000e+01, 1.48760902e+03, 2.50000000e+00, 3.00000000e+00,
       3.99000000e-01])

In [18]:
output_param_names

['Fcapil',
 'aquiferBaseflowExp',
 'aquiferBaseflowRate',
 'frozenPrecipMultip',
 'heightCanopyBottom',
 'heightCanopyTop',
 'k_macropore',
 'k_soil',
 'qSurfScale',
 'routingGammaScale',
 'routingGammaShape',
 'summerLAI',
 'theta_sat']